In [1]:
import pandas as pd
from minsearch import Index

df = pd.read_csv('../data/gold/data.csv')
documents = df.to_dict(orient='records')

In [2]:
index = Index(
    text_fields=["chapter", "title", "section", "text"],
    keyword_fields=[]
)


index.fit(documents)

In [3]:
query = 'what is the scope of a data scientist?'

index.search(query, num_results = 2)

[{'chapter': 'CHAPTER 1',
  'title': 'Machine Learning Roles and the Interview Process',
  'section': 'A Brief History of Machine Learning and Data Science Job Titles',
  'text': 'First, let’s walk through a brief history of job titles. I decided to start with this section to dispel some myths about the “data scientist” job title and shed some light on why there are so many ML-related job titles. After understanding this history, you should be more aware of what job titles to aim for yourself. If you’ve ever been confused about the litany of titles such as machine learning engineer (MLE), product data sci‐ entist, MLOps engineer, and more, this section is for you. ML techniques aren’t a new thing; in 1985, David Ackley, Geoffrey E. Hinton, and Terrence J. Sejnowski popularized the Boltzmann Machine algorithm. 3 Even before that, regression techniques 4 had early developments in the 1800s. There have long been jobs and roles that use modeling techniques to forecast and predict. Econome‐

In [4]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

In [5]:
model_name = "meta-llama/Llama-2-7b"  # Cambia según el modelo que estés usando
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

OSError: Can't load tokenizer for 'meta-llama/Llama-2-7b'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Llama-2-7b' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.

In [ ]:
model.eval()

In [ ]:
def generate_answer(question, context, max_new_tokens=50):
    # Preparar la entrada
    prompt = f"You are an assistant preparing a candidate for a data science interview. " \
             f"Based on the provided context, please provide a concise and accurate answer to the following question. " \
             f"Your response should be solely the answer, without including the question or any context.\n\n" \
             f"QUESTION: {question}\n\n" \
             f"CONTEXT: {context}\n"

    # Tokenizar la entrada
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)

    # Generación de texto
    with torch.no_grad():
        output_tokens = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1,
            temperature=0.7,
            max_new_tokens=max_new_tokens
        )

    # Decodificar los tokens generados
    new_tokens = output_tokens[0][len(inputs['input_ids'][0]):]
    generated_text = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

    return generated_text

In [ ]:
question = "What is the scope of a data scientist?"
context = "First, let’s walk through a brief history of job titles..."

answer = generate_answer(question, context)

In [ ]:
print("Generated Answer:", answer)

In [7]:
import ollama

# Inicializa el cliente Ollama
client = ollama.Client()

# Función para generar respuestas utilizando el modelo Llama2
def generate_answer(question, context, max_new_tokens=50):
    prompt = (
        "You are an assistant preparing a candidate for a data science interview. "
        "Based on the provided context, please provide a concise and accurate answer to the following question. "
        "Your response should be solely the answer, without including the question or any context.\n\n"
        f"QUESTION: {question}\n\n"
        f"CONTEXT: {context}\n"
    )

    # Generar texto
    output = client.generate(prompt)

    return output['text'].strip()

# Ejemplo de uso
if __name__ == "__main__":
    question = "What is the scope of a data scientist?"
    context = "First, let’s walk through a brief history of job titles..."
    
    answer = generate_answer(question, context)
    print("Generated Answer:", answer)

ResponseError: open /usr/share/ollama/.ollama/models/manifests/registry.ollama.ai/library/You are an assistant preparing a candidate for a data science interview. Based on the provided context, please provide a concise and accurate answer to the following question. Your response should be solely the answer, without including the question or any context.

QUESTION/ What is the scope of a data scientist?

CONTEXT: First, let’s walk through a brief history of job titles...
: file name too long